In [7]:
import requests

API_KEY = "YOUR_API_KEY"
BASE_URL = "https://llm-gateway.heurist.xyz/chat/completions"

def query_heurist_llm(prompt, model="qwen/qwen-2.5-coder-32b-instruct"):
    """Query the Qwen LLM for an answer."""
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 500
    }

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(BASE_URL, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json().get("choices", [{}])[0].get("message", {}).get("content", "No response")
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
question = "Does a babygirl has XY chromosome?"
llm_answer = query_heurist_llm(question)
print("LLM Answer:", llm_answer)

LLM Answer: No, a baby girl does not have XY chromosomes. Girls have XX chromosomes. The X and Y chromosomes are sex chromosomes; females inherit one X chromosome from each parent, resulting in XX, while males inherit one X from their mother and one Y from their father, resulting in XY.


In [8]:
!pip install duckduckgo-search

In [9]:
from duckduckgo_search import DDGS

def search_duckduckgo(query, max_results=3):
    """Fetch search results from DuckDuckGo."""
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))
    return results if results else [{"title": "No results", "link": "", "snippet": "No relevant information found."}]

# Get search results
duckduckgo_results = search_duckduckgo(question)
print("DuckDuckGo Results:", duckduckgo_results)

DuckDuckGo Results: [{'title': 'What Are Sex Chromosomes (XX and XY Chromosomes)? - WebMD', 'href': 'https://www.webmd.com/sex/xx-and-xy-chromosomes', 'body': "Sex chromosomes are the genetic elements that define a person's biological sex. Learn about the XX and XY chromosomes, the sex determination system, and other chromosome combinations."}, {'title': 'How Is Your Sex Determined? | Ask A Biologist', 'href': 'https://askabiologist.asu.edu/embryo-tales/sex-chromosomes', 'body': 'People usually have two sex chromosomes. Typically, embryos with one X and one Y chromosome develop into males. And those with two X chromosomes usually become females. We get one sex chromosome from our fathers, carried by sperm, and one from our mothers, carried by the egg.'}, {'title': "New Study Shows Why X And Y Chromosomes Alone Don't Determine a Baby's ...", 'href': 'https://www.sciencealert.com/a-baby-s-sex-is-about-more-than-just-its-x-and-y-chromosomes-new-research-reveals', 'body': 'In the standard 

In [10]:
import json

def query_deepseek_llm(question, llm_answer, duckduckgo_results):
    """Use DeepSeek LLM to compare the answer with DuckDuckGo search results."""
    comparison_prompt = f"""
    The LLM answered: "{llm_answer}"
    DuckDuckGo search results provided the following snippets:
    {json.dumps(duckduckgo_results, indent=2)}

    Based on these, classify the LLM's response as one of the following:
    - "Correct" if it matches DuckDuckGo results
    - "Uncertain" if there's partial confirmation
    - "Hallucinated" if it's not supported at all
    Provide only the classification as output.
    """

    return query_heurist_llm(comparison_prompt, model="deepseek/deepseek-r1")

# Compare the LLM answer
deepseek_verification = query_deepseek_llm(question, llm_answer, duckduckgo_results)
print("DeepSeek Comparison:", deepseek_verification)

DeepSeek Comparison: <think>
Okay, let's check the LLM's answer against the search results. The LLM said that a baby girl has XX chromosomes and explained that females inherit one X from each parent, while males get X from mom and Y from dad.

Looking at the first DuckDuckGo result from WebMD, it mentions that XX and XY are sex chromosomes, with females typically being XX and males XY. That matches the LLM's statement.

The second snippet from Ask A Biologist confirms that two X chromosomes usually result in female development, and one X and one Y in male. Again, this aligns with the LLM's response.

The third link from ScienceAlert adds that typically XX becomes a girl and XY a boy, though other genes can interfere. The LLM didn't mention exceptions, but the core statement about XX and XY is consistent.

All three sources support the basic claim that girls have XX and boys have XY. The LLM's answer doesn't present conflicting information, so it should be classified as correct.
</think